<a href="https://colab.research.google.com/github/armandossrecife/mysentimentanalysis/blob/main/my_automatic_inspection_issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A) Testes de Inspeção de Issues do Cassandra

## Dependencies

### Install dependencies

- datasets from Hugging Face
- transformers Hugging Face
- torch
- accelerate
- ntlk

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip -q install transformers[torch]

In [ ]:
!pip -q install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip -q install nltk

### Import dependencies


- torch
- pandas
- numpy
- transformers
- sklearn
- datasets
- json
- string
- nltk

In [ ]:
import torch
import pandas as pd
import numpy as np

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from datasets import load_dataset
import json

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from urllib.parse import urlparse

### NLTK Dependencies

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Support Functions

In [ ]:
def truncate_string(text, max_length=100, add_ellipsis=True):
  if len(text) <= max_length:
    return text

  truncated_text = text[:max_length]

  if add_ellipsis:
    truncated_text += "..."

  return truncated_text

def to_lowercase(text):
  return text.lower()

def remove_hyperlinks(text):
  tokens = nltk.word_tokenize(text)
  filtered_tokens = [token for token in tokens if not urlparse(token).scheme]
  return ' '.join(filtered_tokens)

def remove_punctuation(text):
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word not in stop_words]
  return ' '.join(filtered_words)

def preprocess_text(text):
  text = to_lowercase(text)
  text = remove_hyperlinks(text)
  #text = remove_punctuation(text)
  text = remove_stopwords(text)
  return text

In [ ]:
coluna_uteis = ['issue_key', 'summary', 'description', 'comments', 'architectural_impact']

## My Hugging Face Dataset

https://huggingface.co/datasets/armandoufpi/cassandraissuesgroundtruth


In [ ]:
splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df_treino = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["train"])
df_teste = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Carrega o dataset e faz os devidos processamentos (transformações)

In [ ]:
dataset = pd.concat([df_treino, df_teste], axis=0)
dataset['Textual_Type'] = 'AI_Yes'
dataset.loc[dataset['label']==0, 'Textual_Type'] = 'AI_No'
dataset['SummaryDescriptionComments']= dataset.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments_text'],axis=1).values
dataset['processed_text'] = dataset['SummaryDescriptionComments'].apply(preprocess_text)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   issue_key                   226 non-null    object
 1   summary                     226 non-null    object
 2   issue_type                  226 non-null    object
 3   issue_status                226 non-null    object
 4   issue_priority              226 non-null    object
 5   description                 226 non-null    object
 6   comments                    226 non-null    object
 7   architectural_impact        226 non-null    object
 8   comments_text               226 non-null    object
 9   label                       226 non-null    int64 
 10  label_text                  226 non-null    object
 11  Textual_Type                226 non-null    object
 12  SummaryDescriptionComments  226 non-null    object
 13  processed_text              226 non-null    object
dtype

In [ ]:
dataset.head()

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text,Textual_Type,SummaryDescriptionComments,processed_text
0,CASSANDRA-3489,EncryptionOptions should be instantiated,Bug,Resolved,Low,"As the title says, otherwise you get an NPE wh...","['There\'s a bunch of ""if encryption options i...",NO,"There\'s a bunch of ""if encryption options is ...",0,negative,AI_No,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot..."
1,CASSANDRA-16780,Log when writing many tombstones to a partition,Improvement,Resolved,Normal,Log when writing many tombstones to a partitio...,['https://github.com/krummas/cassandra/commits...,NO,https://github.com/krummas/cassandra/commits/m...,0,negative,AI_No,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...
2,CASSANDRA-5426,Redesign repair messages,Improvement,Resolved,Low,Many people have been reporting 'repair hang' ...,['Work in progress is pushed to: https://githu...,YES,https://github.com/yukim/cassandra/commits/542...,1,positive,AI_Yes,Redesign repair messages Many people have been...,redesign repair messages many people reporting...
3,CASSANDRA-5121,system.peers.tokens is empty after node restart,Bug,Resolved,Low,Using a 2 nodes fresh cluster (127.0.0.1 & 127...,"['In StorageService.handleStateNormal, when we...",NO,removeEndpoint should be used instead\n [ju...,0,negative,AI_No,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,Bug,Resolved,Normal,Same problem as with CASSANDRA-11886 - if we t...,['https://github.com/krummas/cassandra/commits...,YES,https://github.com/krummas/cassandra/commits/m...,1,positive,AI_Yes,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...


### Atributos chaves

In [ ]:
dataset[['issue_key', 'summary', 'description', 'comments_text', 'label', 'Textual_Type']]

,issue_key,summary,description,comments_text,label,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated,"As the title says, otherwise you get an NPE wh...","There\'s a bunch of ""if encryption options is ...",0,AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partition,Log when writing many tombstones to a partitio...,https://github.com/krummas/cassandra/commits/m...,0,AI_No
2,CASSANDRA-5426,Redesign repair messages,Many people have been reporting 'repair hang' ...,https://github.com/yukim/cassandra/commits/542...,1,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restart,Using a 2 nodes fresh cluster (127.0.0.1 & 127...,removeEndpoint should be used instead\n [ju...,0,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,Same problem as with CASSANDRA-11886 - if we t...,https://github.com/krummas/cassandra/commits/m...,1,AI_Yes
...,...,...,...,...,...,...
21,CASSANDRA-6706,Duplicate rows returned when in clause has rep...,If a value is repeated within an IN clause the...,"[""That is kind of the intended behavior. Is it...",0,AI_No
22,CASSANDRA-6962,examine shortening path length post-5202,From CASSANDRA-5202 discussion:\n\n{quote}\nDi...,feels pretty error prone. What about keeping t...,0,AI_No
23,CASSANDRA-6972,Throw an ERROR when auto_bootstrap: true and b...,Obviously when this condition exists the node ...,false in their seed configs.' 'Yes the right f...,0,AI_No
24,CASSANDRA-758,support wrapped range queries,we want to support scanning from KeyX to KeyA ...,add wrapped range support + test' '+1 Looks go...,0,AI_No


### Dataset transformado

Dataset que será usado para os testes do Apache Cassanda

In [ ]:
minhas_colunas = ['issue_key', 'SummaryDescriptionComments', 'processed_text', 'Textual_Type']
dataset2 = dataset[minhas_colunas]
dataset2.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes


In [ ]:
dataset2['processed_text'][0]

,processed_text
0,"encryptionoptions instantiated title says , ot..."
0,sstablesinbounds might actually give sstables ...


## Set up the environment for the AI ​​model

### Initial Setup

In [ ]:
model_name = 'distilbert-base-uncased'
device_name = 'cuda'
#device_name = 'cpu'
max_length = 512
cached_model_directory_name = 'distilbert-ehbugs'

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [ ]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### Support class to handle the dataset

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def cap_number(x):
    if x > 1:
      return 1
    elif x < 0:
      return 0
    else:
      return x

def compute_metrics(pred):
    labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    outputs = pred.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

### Cleaning directories

Cleaning directories related to results and **logs**

In [ ]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Setup the arguments for training

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='/content/results',   # output directory
    logging_dir='/content/logs',     # directory for storing logs
    logging_steps=150,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [ ]:
unique_labels = {'AI_Yes', 'AI_No'}
label2id = {'AI_No': 0, 'AI_Yes': 1}
id2label = {0: 'AI_No', 1: 'AI_Yes'}

## Create a StratifiedKFold

Stratified K-Fold cross-validator.

Provides train/test indices to split data in train/test sets.

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = dataset['processed_text'], dataset['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

In [ ]:
X

,processed_text
0,"encryptionoptions instantiated title says , ot..."
1,log writing many tombstones partition log writ...
2,redesign repair messages many people reporting...
3,system.peers.tokens empty node restart using 2...
4,sstablesinbounds might actually give sstables ...
...,...
21,duplicate rows returned clause repeated values...
22,examine shortening path length post-5202 cassa...
23,throw error auto_bootstrap : true bootstrappin...
24,support wrapped range queries want support sca...


In [ ]:
y

,Textual_Type
0,AI_No
1,AI_No
2,AI_Yes
3,AI_No
4,AI_Yes
...,...
21,AI_No
22,AI_No
23,AI_No
24,AI_No


In [ ]:
skf.split(X, y)

<generator object _BaseKFold.split at 0x7ea728b19ee0>

In [ ]:
print(f"Running in {device_name}")

Running in cuda


## Train and Evaluate the Model

In [ ]:
list_my_metrics = list()

In [ ]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i+1}: Train Size {len(train_index)} | Test Size {len(test_index)}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    X_train = [str(i) for i in X_train]
    X_test = [str(i) for i in X_test]

    unique_labels = set(label for label in y_train)
    label2id = {label: id for id, label in enumerate(unique_labels)}
    id2label = {id: label for label, id in label2id.items()}

    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
    test_encodings  = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device_name)
    trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics
    )
    trainer.train()
    trainer.evaluate()
    predicted_results = trainer.predict(test_dataset)
    outputs = predicted_results.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)

    # roc_auc_score(test_labels_encoded, probas)
    folds[i] = {}
    folds[i]['pre'] = precision_score(test_labels_encoded, preds)
    folds[i]['rec'] = recall_score(test_labels_encoded, preds)
    folds[i]['acc'] = accuracy_score(test_labels_encoded, preds)
    folds[i]['auc'] = roc_auc_score(test_labels_encoded, probas)
    folds[i]['f1'] = f1_score(test_labels_encoded, preds)

    print(f"Fold {i+1}=> PRE: {folds[i]['pre']}; REC: {folds[i]['rec']}; ACC: {folds[i]['acc']}; F1S: {folds[i]['f1']}; AUC: {folds[i]['auc']}")

    item_metric = {'Fold':i+1, 'PRE':folds[i]['pre'], 'REC':folds[i]['rec'], 'ACC':folds[i]['acc'], 'F1S':folds[i]['f1'], 'AUC':folds[i]['auc']}
    list_my_metrics.append(item_metric)

Fold 1: Train Size 180 | Test Size 46


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 4=> PRE: 0.0; REC: 0.0; ACC: 0.6666666666666666; F1S: 0.0; AUC: 0.6977777777777777
Fold 5: Train Size 181 | Test Size 45


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Fold 5=> PRE: 0.3333333333333333; REC: 0.06666666666666667; ACC: 0.6444444444444445; F1S: 0.1111111111111111; AUC: 0.54


In [ ]:
df_my_metrics = pd.DataFrame(list_my_metrics)
df_my_metrics

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.000000,0.000000,0.673913,0.000000,0.853763
1,2,0.000000,0.000000,0.688889,0.000000,0.801843
2,3,1.000000,0.066667,0.688889,0.125000,0.688889
3,4,0.000000,0.000000,0.666667,0.000000,0.697778
4,5,0.333333,0.066667,0.644444,0.111111,0.540000


# B) Performing tests with Kafka issues

## Original dataset

In [ ]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/kafka_issues_inspected.xlsx

--2024-08-23 15:30:50--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/kafka_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147840 (144K) [application/octet-stream]
Saving to: ‘kafka_issues_inspected.xlsx’

kafka_issues_inspec 100%[===================>] 144.38K  --.-KB/s    in 0.02s   

2024-08-23 15:30:51 (6.18 MB/s) - ‘kafka_issues_inspected.xlsx’ saved [147840/147840]



In [ ]:
df_kafka_issues_inspected = pd.read_excel('kafka_issues_inspected.xlsx')

## Column adjustments

In [ ]:
coluna_uteis = ['issue_key', 'summary', 'description', 'comments', 'architectural_impact']

In [ ]:
df_kafka_issues_inspected = df_kafka_issues_inspected[coluna_uteis]

## Dataset transformations

Transformações necessárias para ajustar o dataset para os testes

In [ ]:
df_kafka_issues_inspected['label'] = df_kafka_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)

In [ ]:
df_kafka_issues_inspected['Textual_Type'] = 'AI_Yes'
df_kafka_issues_inspected.loc[df_kafka_issues_inspected['label']==0, 'Textual_Type'] = 'AI_No'
df_kafka_issues_inspected['SummaryDescriptionComments']= df_kafka_issues_inspected.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments'],axis=1).values
df_kafka_issues_inspected['processed_text'] = df_kafka_issues_inspected['SummaryDescriptionComments'].apply(preprocess_text)

## Selecting key columns

In [ ]:
minhas_colunas = ['issue_key', 'SummaryDescriptionComments', 'processed_text', 'Textual_Type']
df_kafka_issues_inspected = df_kafka_issues_inspected[minhas_colunas]
df_kafka_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,KAFKA-1253,Implement compression in new producer \n It...,implement compression new producer seems incre...,AI_Yes
1,KAFKA-1250,Add slf4j logging to new producer Currently t...,add slf4j logging new producer currently loggi...,AI_No
2,KAFKA-1498,new producer performance and bug improvements ...,new producer performance bug improvements seen...,AI_No
3,KAFKA-2313,javadoc fix for KafkaConsumer deserialization ...,javadoc fix kafkaconsumer deserialization kafk...,AI_No
4,KAFKA-2123,Make new consumer offset commit API use callba...,make new consumer offset commit api use callba...,AI_Yes


## Cleaning directories

In [ ]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_kafka_issues_inspected['processed_text'], df_kafka_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

In [ ]:
skf.split(X, y)

print(f"Running in {device_name}")

Running in cuda


## Performing tests in the model

In [ ]:
list_my_metrics = list()

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i+1}: Train Size {len(train_index)} | Test Size {len(test_index)}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    X_train = [str(i) for i in X_train]
    X_test = [str(i) for i in X_test]

    unique_labels = set(label for label in y_train)
    label2id = {label: id for id, label in enumerate(unique_labels)}
    id2label = {id: label for label, id in label2id.items()}

    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
    test_encodings  = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device_name)
    trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics
    )
    trainer.train()
    trainer.evaluate()
    predicted_results = trainer.predict(test_dataset)
    outputs = predicted_results.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)

    # roc_auc_score(test_labels_encoded, probas)
    folds[i] = {}
    folds[i]['pre'] = precision_score(test_labels_encoded, preds)
    folds[i]['rec'] = recall_score(test_labels_encoded, preds)
    folds[i]['acc'] = accuracy_score(test_labels_encoded, preds)
    folds[i]['auc'] = roc_auc_score(test_labels_encoded, probas)
    folds[i]['f1'] = f1_score(test_labels_encoded, preds)

    print(f"Fold {i+1}=> PRE: {folds[i]['pre']}; REC: {folds[i]['rec']}; ACC: {folds[i]['acc']}; F1S: {folds[i]['f1']}; AUC: {folds[i]['auc']}")

    item_metric = {'Fold':i+1, 'PRE':folds[i]['pre'], 'REC':folds[i]['rec'], 'ACC':folds[i]['acc'], 'F1S':folds[i]['f1'], 'AUC':folds[i]['auc']}
    list_my_metrics.append(item_metric)

Fold 1: Train Size 143 | Test Size 36


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 1=> PRE: 0.0; REC: 0.0; ACC: 0.5833333333333334; F1S: 0.0; AUC: 0.7492063492063492
Fold 2: Train Size 143 | Test Size 36


...

Fold 4=> PRE: 0.42857142857142855; REC: 0.21428571428571427; ACC: 0.5833333333333334; F1S: 0.2857142857142857; AUC: 0.5974025974025975
Fold 5: Train Size 144 | Test Size 35


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Fold 5=> PRE: 0.0; REC: 0.0; ACC: 0.6; F1S: 0.0; AUC: 0.7482993197278911


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Kafka issues results

In [ ]:
df_my_metrics_kafka_issues = pd.DataFrame(list_my_metrics)
df_my_metrics_kafka_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.000000,0.000000,0.583333,0.000000,0.749206
1,2,0.545455,0.400000,0.611111,0.461538,0.600000
2,3,0.500000,0.071429,0.611111,0.125000,0.616883
3,4,0.428571,0.214286,0.583333,0.285714,0.597403
4,5,0.000000,0.000000,0.600000,0.000000,0.748299


# C) Testes de Inspeção de Issues do Hadoop

## Original Dataset

In [ ]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/hadoop_issues_inspected.xlsx

--2024-08-23 15:31:17--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/hadoop_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137652 (134K) [application/octet-stream]
Saving to: ‘hadoop_issues_inspected.xlsx’

hadoop_issues_inspe 100%[===================>] 134.43K  --.-KB/s    in 0.02s   

2024-08-23 15:31:17 (6.25 MB/s) - ‘hadoop_issues_inspected.xlsx’ saved [137652/137652]



In [ ]:
df_hadoop_issues_inspected = pd.read_excel('hadoop_issues_inspected.xlsx')

In [ ]:
def transform_dataset(coluna_uteis, minhas_colunas, df_x_issues_inspected):
	df_x_issues_inspected = df_x_issues_inspected[coluna_uteis]
	df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)
	df_x_issues_inspected['Textual_Type'] = 'AI_Yes'
	df_x_issues_inspected.loc[df_x_issues_inspected['label']==0, 'Textual_Type'] = 'AI_No'
	df_x_issues_inspected['SummaryDescriptionComments']= df_x_issues_inspected.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments'],axis=1).values
	df_x_issues_inspected['processed_text'] = df_x_issues_inspected['SummaryDescriptionComments'].apply(preprocess_text)
	df_x_issues_inspected = df_x_issues_inspected[minhas_colunas]

	return df_x_issues_inspected

In [ ]:
df_hadoop_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_hadoop_issues_inspected)

<ipython-input-53-8ffca01c0d2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)
<ipython-input-53-8ffca01c0d2c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['Textual_Type'] = 'AI_Yes'
<ipython-input-53-8ffca01c0d2c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
df_hadoop_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,HADOOP-6252,Provide method to determine if a deprecated ke...,provide method determine deprecated key set co...,AI_No
1,HADOOP-6184,Provide a configuration dump in json format. ...,provide configuration dump json format . confi...,AI_Yes
2,HADOOP-6165,Add metadata to Serializations The Serializat...,add metadata serializations serialization fram...,AI_Yes
3,HADOOP-6161,Add get/setEnum to Configuration It would be ...,add get/setenum configuration would useful con...,AI_No
4,HADOOP-6103,Configuration clone constructor does not clone...,configuration clone constructor clone members ...,AI_No


## Cleaning diretories

In [ ]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_hadoop_issues_inspected['processed_text'], df_hadoop_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7ea6f3256650>

## Performing tests in the model


In [ ]:
def perform_tests_in_model(X,y, label2id):
  list_my_metrics = list()
  for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i+1}: Train Size {len(train_index)} | Test Size {len(test_index)}")
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    X_train = [str(i) for i in X_train]
    X_test = [str(i) for i in X_test]

    unique_labels = set(label for label in y_train)
    label2id = {label: id for id, label in enumerate(unique_labels)}
    id2label = {id: label for label, id in label2id.items()}
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
    test_encodings  = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)

    train_labels_encoded = [float(label2id[yi]) for yi in y_train]
    test_labels_encoded  = [float(label2id[yi]) for yi in y_test]

    train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)

    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device_name)
    trainer = Trainer(
	      model=model,
	      args=training_args,
	      train_dataset=train_dataset,
	      eval_dataset=test_dataset,
	      compute_metrics=compute_metrics
	    )
    trainer.train()
    trainer.evaluate()

    predicted_results = trainer.predict(test_dataset)
    outputs = predicted_results.predictions.flatten().tolist()
    probas = [cap_number(x) for x in outputs]
    preds = np.array(np.array(probas) > 0.5, dtype=int)

    # roc_auc_score(test_labels_encoded, probas)
    folds[i] = {}
    folds[i]['pre'] = precision_score(test_labels_encoded, preds)
    folds[i]['rec'] = recall_score(test_labels_encoded, preds)
    folds[i]['acc'] = accuracy_score(test_labels_encoded, preds)
    folds[i]['auc'] = roc_auc_score(test_labels_encoded, probas)
    folds[i]['f1'] = f1_score(test_labels_encoded, preds)

    print(f"Fold {i+1}=> PRE: {folds[i]['pre']}; REC: {folds[i]['rec']}; ACC: {folds[i]['acc']}; F1S: {folds[i]['f1']}; AUC: {folds[i]['auc']}")
    item_metric = {'Fold':i+1, 'PRE':folds[i]['pre'], 'REC':folds[i]['rec'], 'ACC':folds[i]['acc'], 'F1S':folds[i]['f1'], 'AUC':folds[i]['auc']}
    list_my_metrics.append(item_metric)

  return list_my_metrics

In [ ]:
list_my_metrics = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 118 | Test Size 30


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 1=> PRE: 0.0; REC: 0.0; ACC: 0.6666666666666666; F1S: 0.0; AUC: 0.75
Fold 2: Train Size 118 | Test Size 30


...

Step,Training Loss,Validation Loss


Fold 4=> PRE: 0.8; REC: 0.4; ACC: 0.7586206896551724; F1S: 0.5333333333333333; AUC: 0.731578947368421
Fold 5: Train Size 119 | Test Size 29


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Fold 5=> PRE: 0.42857142857142855; REC: 0.3; ACC: 0.6206896551724138; F1S: 0.3529411764705882; AUC: 0.5526315789473685


## Hadoop issues results


In [ ]:
df_my_metrics_hadoop_issues = pd.DataFrame(list_my_metrics)
df_my_metrics_hadoop_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.000000,0.0,0.666667,0.000000,0.750000
1,2,0.000000,0.0,0.666667,0.000000,0.610000
2,3,0.000000,0.0,0.633333,0.000000,0.779904
3,4,0.800000,0.4,0.758621,0.533333,0.731579
4,5,0.428571,0.3,0.620690,0.352941,0.552632


# D) Testes de Inspeção de Issues do ActiveMQ

## Original Dataset

In [ ]:
!wget https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/activemq_issues_inspected.xlsx

df_activemq_issues_inspected = pd.read_excel('activemq_issues_inspected.xlsx')

--2024-08-23 15:31:36--  https://raw.githubusercontent.com/armandossrecife/mysentimentanalysis/main/activemq_issues_inspected.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93147 (91K) [application/octet-stream]
Saving to: ‘activemq_issues_inspected.xlsx’

activemq_issues_ins 100%[===================>]  90.96K  --.-KB/s    in 0.02s   

2024-08-23 15:31:36 (5.56 MB/s) - ‘activemq_issues_inspected.xlsx’ saved [93147/93147]



In [ ]:
df_activemq_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_activemq_issues_inspected)

<ipython-input-53-8ffca01c0d2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_issues_inspected['label'] = df_x_issues_inspected['architectural_impact'].apply(lambda x: 1 if x == 'YES' else 0)


In [ ]:
df_activemq_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,AMQ-2149,Shared Filesystem Master Slave: missing messag...,shared filesystem master slave : missing messa...,AI_Yes
1,AMQ-2128,close() from MessageListener.onMessage() with ...,close ( ) messagelistener.onmessage ( ) auto d...,AI_No
2,AMQ-2106,Allow broker to evenly distribute message grou...,allow broker evenly distribute message groups ...,AI_Yes
3,AMQ-2103,Memory leak when marshaling ActiveMQTextMessag...,memory leak marshaling activemqtextmessage per...,AI_No
4,AMQ-2087,Redelivery after rollback does not seem to wor...,redelivery rollback seem work well test case r...,AI_No


## Cleaning diretories

In [ ]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_activemq_issues_inspected['processed_text'], df_activemq_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7ea6ec927060>

## Performing tests in model

In [ ]:
list_my_metrics_activemq = perform_tests_in_model(X,y, label2id=label2id)

Fold 1: Train Size 105 | Test Size 27


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Fold 1=> PRE: 0.42105263157894735; REC: 0.7272727272727273; ACC: 0.48148148148148145; F1S: 0.5333333333333333; AUC: 0.5056818181818181
Fold 2: Train Size 105 | Test Size 27


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss,Validation Loss


Fold 4=> PRE: 0.4166666666666667; REC: 0.45454545454545453; ACC: 0.5; F1S: 0.43478260869565216; AUC: 0.4727272727272728
Fold 5: Train Size 106 | Test Size 26


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Fold 5=> PRE: 0.4166666666666667; REC: 0.45454545454545453; ACC: 0.5; F1S: 0.43478260869565216; AUC: 0.5636363636363636


## ActiveMQ issues results

In [ ]:
df_my_metrics_activemq_issues = pd.DataFrame(list_my_metrics_activemq)
df_my_metrics_activemq_issues

,Fold,PRE,REC,ACC,F1S,AUC
0,1,0.421053,0.727273,0.481481,0.533333,0.505682
1,2,0.388889,0.636364,0.444444,0.482759,0.409091
2,3,0.333333,0.454545,0.384615,0.384615,0.393939
3,4,0.416667,0.454545,0.500000,0.434783,0.472727
4,5,0.416667,0.454545,0.500000,0.434783,0.563636


# E) Performing tests with Kafka, Hadoop and ActiveMQ

## Dataset

In [ ]:
dataset = pd.concat([df_treino, df_teste], axis=0)

In [ ]:
df_cassandra_issues_inspected = dataset[['issue_key', 'summary', 'description', 'comments_text', 'architectural_impact']]


In [ ]:
df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)


<ipython-input-62-6ee6c17d5e9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cassandra_issues_inspected.rename(columns={'comments_text': 'comments'}, inplace=True)


In [ ]:
df_cassandra_issues_inspected = transform_dataset(coluna_uteis, minhas_colunas, df_cassandra_issues_inspected)

In [ ]:
df_cassandra_issues_inspected.head()

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes


## Faz o merge dos datases

Cassandra, Kafka, Hadoop e ActiveMQ

In [ ]:
df_all_issues_inspected = pd.concat([df_cassandra_issues_inspected,df_kafka_issues_inspected, df_hadoop_issues_inspected, df_activemq_issues_inspected], axis=0)


In [ ]:
df_all_issues_inspected

,issue_key,SummaryDescriptionComments,processed_text,Textual_Type
0,CASSANDRA-3489,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot...",AI_No
1,CASSANDRA-16780,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...,AI_No
2,CASSANDRA-5426,Redesign repair messages Many people have been...,redesign repair messages many people reporting...,AI_Yes
3,CASSANDRA-5121,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...,AI_No
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...,AI_Yes
...,...,...,...,...
127,AMQ-8413,Support different username and password for re...,support different username password remote bro...,AI_No
128,AMQ-9202,Reentrant locks should always be locked outsid...,reentrant locks always locked outside try bloc...,AI_No
129,AMQ-9157,Add a new advisory type for dispatched message...,add new advisory type dispatched messages use ...,AI_No
130,AMQ-9153,Fix Slow Consumer Advisory for Queue subscript...,fix slow consumer advisory queue subscriptions...,AI_No


## Cleaning diretories

In [ ]:
!echo 'Cria a pasta results'
!rm -rf results
!mkdir results
!echo 'Cria a pasta logs'
!rm -rf logs
!mkdir logs

Cria a pasta results
Cria a pasta logs


## Create a StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
X, y = df_all_issues_inspected['processed_text'], df_all_issues_inspected['Textual_Type']
skf.get_n_splits(X, y)
folds = {}

skf.split(X, y)

<generator object _BaseKFold.split at 0x7e5141a9f8b0>

## Performing tests in the model

In [ ]:
list_my_metrics_all_repositories = perform_tests_in_model(X,y, label2id=label2id)

# F) Create the Model (based on example from Hugging face DestilBert)

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
task = "issue-analysis"
MY_HUGGING_FACE_DATASET = "armandoufpi/cassandraissuesgroundtruth"

In [ ]:
# Load pre-trained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Dataset da minha conta Hugging Fase

https://huggingface.co/datasets/armandoufpi/cassandraissuesgroundtruth

In [ ]:
#dataset da minha conta Hugging Fase
splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df_treino = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["train"])
df_teste = pd.read_json("hf://datasets/armandoufpi/cassandraissuesgroundtruth/" + splits["test"])

In [ ]:
df_treino['SummaryDescriptionComments']= df_treino.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments_text'],axis=1).values
df_treino['processed_text'] = df_treino['SummaryDescriptionComments'].apply(preprocess_text)

df_teste['SummaryDescriptionComments']= df_teste.apply(lambda row: row['summary'] + ' ' + row['description'] + ' ' + row['comments_text'],axis=1).values
df_teste['processed_text'] = df_teste['SummaryDescriptionComments'].apply(preprocess_text)

In [ ]:
df_treino

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text,SummaryDescriptionComments,processed_text
0,CASSANDRA-3489,EncryptionOptions should be instantiated,Bug,Resolved,Low,"As the title says, otherwise you get an NPE wh...","['There\'s a bunch of ""if encryption options i...",NO,"There\'s a bunch of ""if encryption options is ...",0,negative,EncryptionOptions should be instantiated As th...,"encryptionoptions instantiated title says , ot..."
1,CASSANDRA-16780,Log when writing many tombstones to a partition,Improvement,Resolved,Normal,Log when writing many tombstones to a partitio...,['https://github.com/krummas/cassandra/commits...,NO,https://github.com/krummas/cassandra/commits/m...,0,negative,Log when writing many tombstones to a partitio...,log writing many tombstones partition log writ...
2,CASSANDRA-5426,Redesign repair messages,Improvement,Resolved,Low,Many people have been reporting 'repair hang' ...,['Work in progress is pushed to: https://githu...,YES,https://github.com/yukim/cassandra/commits/542...,1,positive,Redesign repair messages Many people have been...,redesign repair messages many people reporting...
3,CASSANDRA-5121,system.peers.tokens is empty after node restart,Bug,Resolved,Low,Using a 2 nodes fresh cluster (127.0.0.1 & 127...,"['In StorageService.handleStateNormal, when we...",NO,removeEndpoint should be used instead\n [ju...,0,negative,system.peers.tokens is empty after node restar...,system.peers.tokens empty node restart using 2...
4,CASSANDRA-11944,sstablesInBounds might not actually give all s...,Bug,Resolved,Normal,Same problem as with CASSANDRA-11886 - if we t...,['https://github.com/krummas/cassandra/commits...,YES,https://github.com/krummas/cassandra/commits/m...,1,positive,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,CASSANDRA-18617,Disable the deprecated keyspace/table threshol...,Improvement,Resolved,Normal,The non-guardrail thresholds 'keyspace_count_w...,"[""Part of this change is to add converters tha...",YES,\xa0[https://github.com/apache/cassandra/pull/...,1,positive,Disable the deprecated keyspace/table threshol...,disable deprecated keyspace/table thresholds c...
196,CASSANDRA-5244,Compactions don't work while node is bootstrap...,Bug,Resolved,Urgent,It seems that there is a race condition in Sto...,"[""Thanks for the detective work, Jouni. I'll ...",NO,BLOCKED (on object monitor)\n at org.apache...,0,negative,Compactions don't work while node is bootstrap...,compactions n't work node bootstrapping seems ...
197,CASSANDRA-173,add getPendingTasks to CFSMBean,Improvement,Resolved,Low,need to add an atomicint and inc/decr it whene...,['rebased patch as 0001-CASSANDRA-173-added-CF...,NO,rebased patch as 0001-CASSANDRA-173-added-CFS-...,0,negative,add getPendingTasks to CFSMBean need to add an...,add getpendingtasks cfsmbean need add atomicin...
198,CASSANDRA-359,CFS readStats_ and diskReadStats_ are missing,Bug,Resolved,Normal,There is no description,"[""shouldn't we also get rid of getReadDiskHits...",NO,"[""shouldn't we also get rid of getReadDiskHits...",0,negative,CFS readStats_ and diskReadStats_ are missing ...,cfs readstats_ diskreadstats_ missing descript...


In [ ]:
df_teste

,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact,comments_text,label,label_text,SummaryDescriptionComments,processed_text
0,CASSANDRA-11944,sstablesInBounds might not actually give all s...,Bug,Resolved,Normal,Same problem as with CASSANDRA-11886 - if we t...,['https://github.com/krummas/cassandra/commits...,YES,https://github.com/krummas/cassandra/commits/m...,1,positive,sstablesInBounds might not actually give all s...,sstablesinbounds might actually give sstables ...
1,CASSANDRA-12988,make the consistency level for user-level auth...,Improvement,Resolved,Low,Most reads for the auth-related tables execute...,['Linked patch allows an operator to set the r...,YES,[Link|https://app.circleci.com/pipelines/githu...,1,positive,make the consistency level for user-level auth...,make consistency level user-level auth reads w...
2,CASSANDRA-15004,Anti-compaction briefly corrupts sstable state...,Bug,Resolved,Urgent,Since we use multiple sstable rewriters in ant...,['|[3.0|https://github.com/bdeggleston/cassand...,YES,not sure what is going on with the dtests thou...,1,positive,Anti-compaction briefly corrupts sstable state...,anti-compaction briefly corrupts sstable state...
3,CASSANDRA-15265,Index summary redistribution can start even wh...,Bug,Resolved,Normal,When we pause autocompaction for upgradesstabl...,['Patch adds a flag in `CompactionManager` whi...,YES,[3.0|https://circleci.com/workflow-run/8882a8a...,1,positive,Index summary redistribution can start even wh...,index summary redistribution start even compac...
4,CASSANDRA-18029,fix starting Paxos auto repair,Bug,Resolved,Normal,This test was not run in CI because of its nam...,['I fixed here what I could: [https://github.c...,YES,repaired}} rely on running regular/incremental...,1,positive,fix starting Paxos auto repair This test was n...,fix starting paxos auto repair test run ci nam...
5,CASSANDRA-18058,In-memory index and query path,New Feature,Resolved,Normal,An in-memory index using the in-memory trie st...,['The github PR for this ticket is here:\xa0\r...,YES,[https://app.circleci.com/pipelines/github/ade...,1,positive,In-memory index and query path An in-memory in...,in-memory index query path in-memory index usi...
6,CASSANDRA-18617,Disable the deprecated keyspace/table threshol...,Improvement,Resolved,Normal,The non-guardrail thresholds 'keyspace_count_w...,"[""Part of this change is to add converters tha...",YES,\xa0[https://github.com/apache/cassandra/pull/...,1,positive,Disable the deprecated keyspace/table threshol...,disable deprecated keyspace/table thresholds c...
7,CASSANDRA-1919,Add shutdownhook to flush commitlog,Improvement,Resolved,Low,this replaces the periodic_with_flush approach...,"[""The approach I took was to add a shutdownBlo...",YES,Could not create ServerSocket on address /127....,1,positive,Add shutdownhook to flush commitlog this repla...,add shutdownhook flush commitlog replaces peri...
8,CASSANDRA-414,remove sstableLock,Improvement,Resolved,Normal,There is no description,['rebased.\n\n02\n remove sstableLock. re-...,YES,the cleanup does happen. If it were the SSTR ...,1,positive,remove sstableLock There is no description the...,remove sstablelock description cleanup happen ...
9,CASSANDRA-5426,Redesign repair messages,Improvement,Resolved,Low,Many people have been reporting 'repair hang' ...,['Work in progress is pushed to: https://githu...,YES,https://github.com/yukim/cassandra/commits/542...,1,positive,Redesign repair messages Many people have been...,redesign repair messages many people reporting...


## Carrega os dados de treino e teste

In [ ]:
# Load the dataset from armandoufpi hugging face
train_data = load_dataset(MY_HUGGING_FACE_DATASET, split="train")
test_data = load_dataset(MY_HUGGING_FACE_DATASET, split="test")

In [ ]:
train_data

Dataset({
    features: ['summary', 'architectural_impact', 'comments_text', 'label_text', 'comments', 'issue_status', 'description', 'issue_priority', 'issue_type', 'issue_key', 'label'],
    num_rows: 200
})

In [ ]:
test_data

Dataset({
    features: ['summary', 'architectural_impact', 'comments_text', 'label_text', 'comments', 'issue_status', 'description', 'issue_priority', 'issue_type', 'issue_key', 'label'],
    num_rows: 26
})

In [ ]:
print(f"len(train_data['summary']): {len(train_data['summary'])}")
print(f"train_data['summary'][0]: {train_data['summary'][0]}")
print(f"train_data['label'][0]: {train_data['label'][0]}")
print(f"train_data['label_text'][0]: {train_data['label_text'][0]}")
print(f"train_data['description'][0]: {train_data['description'][0]}")

len(train_data['summary']): 200
train_data['summary'][0]: EncryptionOptions should be instantiated
train_data['label'][0]: 0
train_data['label_text'][0]: negative
train_data['description'][0]: As the title says, otherwise you get an NPE when the options are missing from the yaml.  It's included in my second patch on CASSANDRA-3045 and is a one line fix.


## Processa os dados de treino e teste

In [ ]:
# Function to preprocess text data
def preprocess_function_description(examples):
  return tokenizer(examples["description"], padding="max_length", truncation=True)

def preprocess_function_description(examples):
  return tokenizer(examples["description"], padding="max_length", truncation=True)

# Function to preprocess text data
def preprocess_function(examples):
  return tokenizer(examples["summary"], padding="max_length", truncation=True)

In [ ]:
# Preprocess train and test data
train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

In [ ]:
train_data

Dataset({
    features: ['summary', 'architectural_impact', 'comments_text', 'label_text', 'comments', 'issue_status', 'description', 'issue_priority', 'issue_type', 'issue_key', 'label', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
test_data

Dataset({
    features: ['summary', 'architectural_impact', 'comments_text', 'label_text', 'comments', 'issue_status', 'description', 'issue_priority', 'issue_type', 'issue_key', 'label', 'input_ids', 'attention_mask'],
    num_rows: 26
})

In [ ]:
# Access the 'input_ids' from the preprocessed data
#train_inputs = train_data["input_ids"]
#test_inputs = test_data["input_ids"]

## Treina o modelo

In [ ]:
!rm -rf results
!mkdir results
!ls -l

total 12
drwxr-xr-x 2 root root 4096 Jul 30 20:58 logs
drwxr-xr-x 2 root root 4096 Jul 30 21:56 results
drwxr-xr-x 1 root root 4096 Jul 29 13:22 sample_data


In [ ]:
training_args = TrainingArguments(
    output_dir="results",  # Fixed typo (removed extra space)
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Assuming you meant "size" here
    learning_rate=2e-5,
)

In [ ]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics="accuracy",
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=39, training_loss=0.06696320191407815, metrics={'train_runtime': 39.1932, 'train_samples_per_second': 15.309, 'train_steps_per_second': 0.995, 'total_flos': 79480439193600.0, 'train_loss': 0.06696320191407815, 'epoch': 3.0})

## Faz as previsões baseadas no modelo treinado

In [ ]:
# TODO: fazer a analise do issue baseado em varios fields ao mesmo tempo
def analyse_issue(issue_field):
  inputs = tokenizer(issue_field, padding="max_length", truncation=True, return_tensors="pt")

  # Move the model and input to GPU if available
  if torch.cuda.is_available():
    model.to('cuda')
    inputs.to('cuda')

  with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

  # Print the predicted sentiment
  if predictions == 1:
    return "YES"
  else:
    return "NO"

### Dados de testes

In [ ]:
df_teste[['issue_key', 'summary', 'architectural_impact']]

,issue_key,summary,architectural_impact
0,CASSANDRA-11944,sstablesInBounds might not actually give all s...,YES
1,CASSANDRA-12988,make the consistency level for user-level auth...,YES
2,CASSANDRA-15004,Anti-compaction briefly corrupts sstable state...,YES
3,CASSANDRA-15265,Index summary redistribution can start even wh...,YES
4,CASSANDRA-18029,fix starting Paxos auto repair,YES
5,CASSANDRA-18058,In-memory index and query path,YES
6,CASSANDRA-18617,Disable the deprecated keyspace/table threshol...,YES
7,CASSANDRA-1919,Add shutdownhook to flush commitlog,YES
8,CASSANDRA-414,remove sstableLock,YES
9,CASSANDRA-5426,Redesign repair messages,YES


In [ ]:
yes_count = df_teste.architectural_impact.to_list().count("YES")
no_count = df_teste.architectural_impact.to_list().count("NO")

# Print the counts
print("YES count:", yes_count)
print("NO count:", no_count)

YES count: 10
NO count: 16


### Roda o modelo com os dados de testes (baseado apenas no campo Summary)

In [ ]:
lista_analisa_summary_yes_no = []

for index, row in df_teste.iterrows():
  field = row['summary']
  issue_key = row['issue_key']
  summary = truncate_string(text=row['summary'], max_length=50)
  previsao = analyse_issue(issue_field=field)
  print(f"{issue_key}, {summary}, Architectural Impact:{previsao}")
  lista_analisa_summary_yes_no.append(previsao)

CASSANDRA-11944, sstablesInBounds might not actually give all sstab..., Architectural Impact:YES
CASSANDRA-12988, make the consistency level for user-level auth rea..., Architectural Impact:YES
CASSANDRA-15004, Anti-compaction briefly corrupts sstable state for..., Architectural Impact:YES
CASSANDRA-15265, Index summary redistribution can start even when c..., Architectural Impact:YES
CASSANDRA-18029, fix starting Paxos auto repair, Architectural Impact:YES
CASSANDRA-18058, In-memory index and query path, Architectural Impact:YES
CASSANDRA-18617, Disable the deprecated keyspace/table thresholds a..., Architectural Impact:YES
CASSANDRA-1919, Add shutdownhook to flush commitlog, Architectural Impact:YES
CASSANDRA-414, remove sstableLock, Architectural Impact:YES
CASSANDRA-5426, Redesign repair messages, Architectural Impact:YES
CASSANDRA-11540, The JVM should exit if jmx fails to bind, Architectural Impact:NO
CASSANDRA-6013, CAS may return false but still commit the insert, Architectural

In [ ]:
yes_count_summary = lista_analisa_summary_yes_no.count("YES")
no_count_summary = lista_analisa_summary_yes_no.count("NO")

# Print the counts
print("YES count:", yes_count_summary)
print("NO count:", no_count_summary)

YES count: 9
NO count: 17
